<a href="https://colab.research.google.com/github/recervictory/CCMB/blob/master/BWT%20Algorithm%20and%20FM%20Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Burrows-Wheeler Transform and FM Index
by Ben Langmead, Department of Computer Science, JHU

## 1. Burrows-Wheeler Transform

The Burrows-Wheeler Transform (BWT) is a way of permuting the characters of a string T into
another string BW T(T). This permutation is reversible; one procedure exists for turning T into
BW T(T) and another exists for turning BW T(T) back into T. The transformation was originally
discovered by David Wheeler in 1983, and was published by Michael Burrows and David Wheeler
in 1994 [1].
The BWT has two main applications: compression and indexing. We will discuss both. First
we discuss the transformation from T to BW T(T)

### 1.1 BWT via the BWM

T denotes the string we would like to transform, and m = |T| (the length of T). We assume that T
ends with a terminator character, denoted $. We define $ to be a character that does not appear
elsewhere in T, and which is lexicographically prior to all other characters. In the case of DNA
strings, for example, the alphabet order with $ might be $ < A < C < G < T.
Take T = abaaba$. First, we write down the rotations of T: the distinct strings we can make
from T by repeatedly taking a character from one end and sticking it on the other:
```
$ a b a a b a
a $ a b a a b
b a $ a b a a
a b a $ a b a
a a b a $ a b
b a a b a $ a
a b a a b a $
```

By writing them stacked vertically, we’ve created an m × m matrix. Now we sort the rows of
the matrix lexicographically (i.e. alphabetically)

```
$ a b a a b a
a $ a b a a b
a a b a $ a b
a b a $ a b a
a b a a b a $
b a $ a b a a
b a a b a $ a
```

This is the Burrows-Wheeler Matrix (BWM(T)). The final column of BWM(T), read from
top to bottom, is BW T(T). So for T = abaaba$, BW T(T) = abba$aa.

### 1.2 BWT via the suffix array

The Burrows-Wheeler Matrix seems to be related to the suffix array: to make a suffix array of T,
SA(T), we sort T’s suffixes, and to make BWM(T), we sort T’s rotations. The relationship is
clearer when we write them side by side:

```
BWM:        SA: Suffixes given by SA:
$abaaba     6   $
a$abaab     5   a$
aaba$ab     2   aaba$
aba$aba     3   aba$
abaaba$     0   abaaba$
ba$abaa     4   ba$
baaba$a     1   baaba$
```

They correspond to the same ordering. Look at, for example, where the $s appear in each row
of the comparison. So another way of defining BW T(T) is via the suffix array SA(T). Let BW T[i]
denote the character at 0-based offset i in BW T(T) and let SA[i] denote the suffix at 0-based offset
i in SA(T).

```
            { T[SA[i] − 1]  if SA[[i] > 0
BWT[i] =    { $             if SA = 0           
            
```

In [ ]:
def suffixArray(s):
    """ Given T return suffix array SA(T). We use Python’s sorted
    function here for simplicity, but we can do better. """
    # Empty suffix ’’ plays role of $.
    satups = sorted([(s[i:], i) for i in range(0, len(s)+1)])
    # Extract and return just the offsets
    return map(lambda x: x[1], satups)

In [ ]:
def bwt(t):
    """ Given T, returns BWT(T), by way of the suffix array. """
    bw = []
    for si in suffixArray(t):
        if si == 0:
            bw.append('$')
        else:
            bw.append(t[si-1])
    return ''.join(bw) # return string-ized version of list bw

### 1.3 Burrows-Wheeler Transform in compression

How is the Burrows-Wheeler Transform useful for compression? First, it’s reversible. Transformations used in compression must be reversible to allow both compression and decompression.
Second, characters with similar right-contexts in T tend to come togehter in BW T(T). This can,
for instance, bring several occurrences of the same character together in a tight bunch. This is hard
to see in small examples; in the following example, this bunching is more obvious:

In [ ]:
bwt("tomorrow and tomorrow and tomorrow and no more tomorrow")

'wwwwodedd   nnnr ooooaaa nttttmmmmmrrrrorrrroooo   $oooo'

This makes BW T(T) more compressible. For example, we could take BW T(T) and shrink it
(reversibly) with run-length encoding (RLE). Software tools for compression and decompression
employ various methods to shrink BW T(T), including move-to-front transforms, run-length encoding, Huffman coding, and arithmetic coding. The popular bzip2 compression tool [3] uses these
and other methods.

### 1.4 Reversing the Burrows-Wheeler Transform with the LF Mapping

We said the BWT is reversible, but it’s far from obvious at first glance how to reverse it. Recall that
BW T(abaaba$) = abba$aa. It seems at first glance that information about which a in BW T(T)
corresponds to which a in T has been lost.
But the BWT has an important property called the LF Mapping. Consider again the BWM for
T = abaaba$:

```
$ a b a a b a
a $ a b a a b
a a b a $ a b
a b a $ a b a
a b a a b a $
b a $ a b a a
b a a b a $ a
```

We re-write T, this time giving each character (except $) a subscript: `T` = `a0b0a1a2b1a3$`. The
subscript equals the number of times that character occurs previously in T. The first occurrence
of a becomes a0, the second occurrnce of c becomes c1, etc. We call the subscript the character’s
\rank." We don’t rank $ since there’s only one.
Now we re-write the BWM including the ranks. Ranks don’t affect lexicographical order. E.g.
a0 and a999 are equal as far as the ordering of the rows of BWM(T) is concerned.

```
$ a0 b0 a1 a2 b1 a3
a3 $ a0 b0 a1 a2 b1
a1 a2 b1 a3 $ a0 b0
a2 b1 a3 $ a0 b0 a1
a0 b0 a1 a2 b1 a3 $
b1 a3 $ a0 b0 a1 a2
b0 a1 a2 b1 a3 $ a0
```

The LF Mapping states: the ith occurrence of a character c in the last column has the same
rank as the ith occurrence of c in the first column. E.g. look at the as in the last column above,
starting at the top. They have ranks 3, 1, 2 and 0 in that order. Now look at the ranks of the a in
the first column above; they have the same order: 3, 1, 2, 0. Same for the bs.

Why does this property hold? Let M be BWM(T) for some T. Let M0 be the matrix obtained
by rotating all the rows of M to the right by one position. The first column of M0 equals the last
column of M.

Pick any character c and compare the ranks of the cs in the first column of M and the ranks of
the cs in the first column of M0. The ranks appear in the same order because the sort treats those
rows identically; in M the rows are sorted starting at the first position, and in M0 the rows are tied
with respect to their first position and sorted starting at the second position. Because of this, and
because the first column of M0 equals the last column of M, the LF Mapping property follows.

Now let’s see how to reverse the BWT. First, let’s re-rank the characters. Before we ranked
them according to how many times the same character occurred previously in T. Call this the
T-ranking. Now we re-rank according to how many times the same character occurred previously
in BW T(T). Call this the B-ranking. Here is BW T(T) with B-ranks: a0b0b1a1$a2a3.

How are ranks represented? Let’s define an array rank parallel to BW T(T) that stores the
rank of each character. Here is an illustation of the first and last columns of BWM(T), along with
rank.

```
F L rank
$ a  0
a b  0
a b  1
a a  1
a $  0
b a  2
b a  3
```


Informally, to recover T from BW T(T): start in the first row, which must have `$` in the first
column. Rows are rotations of T, so the last column of the first row contains the character to the
left of `$` in T: a in this case. We know from the rank array that this is the a of rank 0: a0. Now
how to recover the character to the left of a0? We can do this if we know which row begins with
a0. But the LF Mapping tells us this. Because a0 has rank 0, it must correspond to the first a in
the first column, i.e. the a in the second row. So we advance to the second row. Now we look in
the last column and rank array and see that b0 precedes a0. b0 corresponds to the first b in the
first column, so we advance to the sixth row. We proceed in this way until we reach the row with
`$` in the last column. In this example, we would visit the rows in this order, assuming the first row
has index 0: (0, 1, 5, 3, 2, 6, 4), and we successfully recreate the original string from right to left:
`a3b1a1a2b0a0$`

In [ ]:
def rankBwt(bw):
    """ Given BWT string bw, returns a parallel list of B-ranks. Also
    returns tots, a mapping from characters to # times the
    character appears in BWT. """
    tots = dict()
    ranks = []
    for c in bw:
        if c not in tots:
            tots[c] = 0
        ranks.append(tots[c])
        tots[c] += 1
    return ranks, tots

def firstCol(tots):
    """ Return a map from characters to the range of cells in the first
    column containing the character. """
    first = {}
    totc = 0
    for c, count in sorted(tots.items()):
        first[c] = (totc, totc + count)
        totc += count
    return first

def reverseBwt(bw):
    """ Make T from BWT(T) """
    ranks, tots = rankBwt(bw)
    #print(ranks, tots)
    first = firstCol(tots)
    print(first)
    rowi = 0
    t = "$"
    while bw[rowi] != '$':
        c = bw[rowi]
        t = c + t
        rowi = first[c][0] + ranks[rowi]
    return t

In [ ]:
bw = bwt("Tomorrow_and_tomorrow_and_tomorrow")
bw

'w$wwdd__nnoooaattTmmmrrrrrrooo__ooo'

In [ ]:
reverseBwt(bw)

{'$': (0, 1), 'T': (1, 2), '_': (2, 6), 'a': (6, 8), 'd': (8, 10), 'm': (10, 13), 'n': (13, 15), 'o': (15, 24), 'r': (24, 30), 't': (30, 32), 'w': (32, 35)}


'Tomorrow_and_tomorrow_and_tomorrow$'

# 2. FM Index

In 2000, six years after the BWT was published, Paolo Ferragina and Giovanni Manzini published
a paper [2] describing how the BWT, together with some small auxilliary data structures, can be
used as a space-efficient index of T. It generally uses less than half the space required to store m integers. They named it the FM Index. Just as the LF Mapping was the key to understanding how
the BWT is reversible, it’s also the key to how it can be used as an index.
Let’s start by considering just the first column (F) and last column (L) of the BWM, as well
as the rank array.

```
F L rank
$ a 0
a b 0
a b 1
a a 1
a $ 0
b a 2
b a 3
```

## 3 Searching
Say we are searching for occurrences of a string P = aba. Like the suffix array, the BWM is sorted.
This implies that any rows having P as a prefix will be consecutive.

We proceed first by finding the rows beginning with the shortest proper suffix of P: a in this
case. The first column is part of our index, so this is trivial. These are the rows in the 0-based
range [1, 5). Let’s visualize this in the context of the whole matrix:
```
F           L  rank
$ a b a a b a   0
a $ a b a a b   0 *****
a a b a $ a b   1 *****
a b a $ a b a   1 *****
a b a a b a $   0 *****
b a $ a b a a   2
b a a b a $ a   3
```

Remember: even though we just drew the entire matrix, our index so far contains just F, L
and rank.
Now we must find all rows beginning with the next-longest proper suffix of P: ba. Observe the
shaded characters in the L above. We see two bs, indicating there are two instances where a is
preceeded by b. Also, the LF Mapping and rank array tell us which rows have ba as a prefix: the
rows beginning with b0 and b1; i.e. the first two rows in the \b section".

```
F           L  rank
$ a b a a b a   0
a $ a b a a b   0 
a a b a $ a b   1 
a b a $ a b a   1 
a b a a b a $   0 
b a $ a b a a   2 *****
b a a b a $ a   3 *****
```

Now we find rows beginning with the final suffix, aba. Again we look at the shaded characters
in the last column. We see that the occurrences of ba are preceded by a2 and a3, giving us the
range of rows prefixed by P:

```
F           L  rank
$ a b a a b a   0
a $ a b a a b   0 
a a b a $ a b   1 
a b a $ a b a   1 *****
a b a a b a $   0 *****
b a $ a b a a   2 
b a a b a $ a   3
```

his is called backwards matching. In short, we apply the LF Mapping repeatedly to find the
range of rows prefixed by successively longer proper suffixes of P until the range becomes empty,
in which case P does not occur in T, or until we run out of suffixes. If we run out of suffixes, the
size of the range equals the number of times P occurs in T.

#### Here is a Python implementation:

In [ ]:
def countMatches(bw, p):
    """ Given BWT(T) and a pattern string p, return the number of times
    p occurs in T. """
    ranks, tots = rankBwt(bw)
    first = firstCol(tots)
    l, r = first[p[-1]]
    i = len(p)-2
    while i >= 0 and r > l:
        c = p[i]
        # scan from left, looking for occurrences of c
        j = l
        while j < r:
            if bw[j] == c:
                l = first[c][0] + ranks[j]
                break
            j += 1
        if j == r:
            l = r
            break # no occurrences -> no match
        r -= 1
        while bw[r] != c:
            r -= 1
        r = first[c][0] + ranks[r] + 1
        i -= 1
    return r - l

In [ ]:
bw = bwt("AAATTTTCCCGGGAAAGGGCCTATATAGGATATACATA")
bw

'ATG$AATTACTTGTAATCGCCGGGGAGCAAAAAACTTTA'

In [ ]:
countMatches(bw, "TATATA")

1